In [109]:
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

from matplotlib import pyplot as plt
%pylab inline

import base64
import json
import time
import urllib


#hostname='haswell01.eng.platformlab.ibm.com'
hostname='wml1x184.eng.platformlab.ibm.com'
commonHeaders={'accept': 'application/json'}
req = requests.Session()

Populating the interactive namespace from numpy and matplotlib


In [110]:
getTuneStatusUrl = 'https://{}:49443/platform/rest/deeplearning/v1/hypersearch'.format(hostname)
r = req.get(getTuneStatusUrl, headers=commonHeaders, verify=False, auth=('Admin', 'Admin'))
if not r.ok:
    print('check hpo task status failed: code=%s, %s'%(res.status_code, res.content))
else:
    if len(r.json()) == 0:
        print('There is no hpo task been created')
    for item in r.json():
        print('Hpo task: %s, State: %s'%(item['hpoName'], item['state']))

Hpo task: Admin-hpo-9551207096118021, State: FINISHED


In [111]:
import tarfile
import tempfile
import os
def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))
MODEL_DIR_SUFFIX = ".modelDir.tar"
tempFile = tempfile.mktemp(MODEL_DIR_SUFFIX)
#make_tarfile(tempFile, '/Users/donglin/Documents/office/work/DLI/design/HPO/kyw/tf-model')
make_tarfile(tempFile, '/Users/jysjiang/Documents/WMLA/RFE/Wells-Fargo-WML-A/tf-model/')
print(tempFile)
files = {'file': open(tempFile, 'rb')}

/var/folders/sm/_vlrb66910jg32402zz_j6mw0000gp/T/tmpkunixa3w.modelDir.tar


In [112]:
data =  {
        'modelSpec': # Define the model training related parameters
        {
            # Spark instance group which will be used to run the HPO sub-trainings. The Spark instance group selected
            # here should match the sub-training args, for example, if the sub-training args try to run a EDT job,
            # then we should put a Spark instance group with capability to run EDT job here.
            'sigName': 'sig0',
            
            # These are the arguments we'll pass to the execution engine; they follow the same conventions
            # of the dlicmd.py command line launcher
            #
            # See:
            #   https://www.ibm.com/support/knowledgecenter/en/SSFHA8_1.2.1/cm/dlicmd.html
            # In this example, args after --model-dir are all the required parameter for the original model itself.
            'args': '--exec-start tensorflow --python-version 2.7 --cs-datastore-meta type=fs \
                     --gpuPerWorker 1 --model-main convolutional_network.py \
                     --trainImagesFile /dlfs/autotest_testgrid/dataset/wmla_data/tf-mnist/train-images-idx3-ubyte.gz \
                     --trainLabelsFile /dlfs/autotest_testgrid/dataset/wmla_data/tf-mnist/train-labels-idx1-ubyte.gz \
                     --testImagesFile /dlfs/autotest_testgrid/dataset/wmla_data/tf-mnist/t10k-images-idx3-ubyte.gz \
                     --testLabelsFile /dlfs/autotest_testgrid/dataset/wmla_data/tf-mnist/t10k-labels-idx1-ubyte.gz'
        },
        'algoDef': # Define the parameters for search algorithms
        {
            # Name of the search algorithm, one of Random, Bayesian, Tpe, Hyperband
            'algorithm': 'ExperimentGridSearch', 
            # Max running time of the hpo task in minutes, -1 means unlimited
            'maxRunTime': -1,  
            # Max number of training job to submitted for hpo task, -1 means unlimited’,
            'maxJobNum': 2,            
            # Max number of training job to run in parallel, default 1. It depends on both the
            # avaiable resource and if the search algorithm support to run in parallel, current only Random
            # fully supports to run in parallel, Hyperband and Tpe supports to to in parellel in some phase,
            # Bayesian runs in sequence now.
            'maxParalleJobNum': 1, 
            # Name of the target metric that we are trying to optimize when searching hyper-parameters.
            # It is the same metric name that the model update part 2 trying to dump.
            'objectiveMetric' : 'loss',
            # Strategy as how to optimize the hyper-parameters, minimize means to find better hyper-parameters to
            # make the above objectiveMetric as small as possible, maximize means the opposite.
            'objective' : 'minimize',
            # eta value to control the hyper-band search process
            'hyperbandEta': 3.0,
            #Additional parameters for the specified search algorithm and hyper-band get following too.
            'algoParams' : 
                [
                    {
                        # Name of the the maximum amount of resource that can be allocated to a single configuration
                        'name':'ResourceName', 
                        'value': 'epochs'
                    },
                    {
                        # Value of the the maximum amount of resource that can be allocated to a single configuration
                        'name':'ResourceValue',
                        'value':'10'
                    }
                    # This resource parameter will change during hyperband searching phase and its value will be put
                    # into the config.json for each sub-training too, here with the 'epoch' as key and value in the
                    # range 1-10.
                ]
        },
    
        # Define the hyper-paremeters to search and the corresponding search space.            
        'experiments':
        [
             {
                 'id':1,
                 'hyperParams': [
                     {
                         'name': 'learning_rate',
                         'dataType': 'DOUBLE',
                         'fixedVal': '0.1'
                     },
                     {
                         'name': 'optimizer',
                         'dataType': 'STRING',
                         'fixedVal': 'Adam'
                     },
                     {
                         'name': 'opt_decay',
                         'dataType': 'DOUBLE',
                         'fixedVal': '0.4882855458519536'
                     },
                     {
                         'name': 'beta1',
                         'dataType': 'DOUBLE',
                         'fixedVal': '0.7'
                     },
                     {
                         'name': 'beta2',
                         'dataType': 'DOUBLE',
                         'fixedVal': '0.8982816529517174'
                     }
                 ]
             },
            {
                 'id':2,
                 'hyperParams': [
                     {
                         'name': 'learning_rate',
                         'dataType': 'DOUBLE',
                         'fixedVal': '0.2'
                     },
                     {
                         'name': 'optimizer',
                         'dataType': 'STRING',
                         'fixedVal': 'Adam'
                     }
                 ]
             }
        ]
       }
mydata={'data':json.dumps(data)}

In [113]:
startTuneUrl='https://{}:49443/platform/rest/deeplearning/v1/hypersearch'.format(hostname)
create = req.post(startTuneUrl, headers=commonHeaders, data=mydata, files=files, verify=False, auth=('Admin', 'Admin'))
if not create.ok:
   print('submit tune job failed: code=%s, %s'%(create.status_code, create.content))
else:
   print('submit tune job succeed with hponame: %s'%create.json())

submit tune job succeed with hponame: Admin-hpo-9559865415941081


In [114]:
import time
#Here we just use the hpo name returned of the previous POST call
hpoName = create.json()

getHpoUrl = 'https://{}:49443/platform/rest/deeplearning/v1/hypersearch/{}'.format(hostname, hpoName)
res = req.get(getHpoUrl, headers=commonHeaders, verify=False, auth=('Admin', 'Admin'))
if not res.ok:
    print('get hpo task failed: code=%s, %s'%(res.status_code, res.content))
else:
    json_out=res.json()
    
    while json_out['state'] in ['SUBMITTED','RUNNING']:
        print('Hpo task {} state {} progress {}'.format(hpoName, json_out['state'], json_out['progress']))
        res = req.get(getHpoUrl, headers=commonHeaders, verify=False, auth=('Admin', 'Admin'))
        json_out=res.json()
        time.sleep(20)
    
    print('Hpo task %s completes with state %s'%(hpoName, json_out['state']))
    #print('Best:%s'%json.dumps(json_out.get('best'), sort_keys=True, indent=4))
    print(json.dumps(res.json(), sort_keys=True, indent=4))

Hpo task Admin-hpo-9559865415941081 state RUNNING progress 0/2
Hpo task Admin-hpo-9559865415941081 state RUNNING progress 0/2
Hpo task Admin-hpo-9559865415941081 state RUNNING progress 0/2
Hpo task Admin-hpo-9559865415941081 state RUNNING progress 0/2
Hpo task Admin-hpo-9559865415941081 state RUNNING progress 1/2
Hpo task Admin-hpo-9559865415941081 state RUNNING progress 1/2
Hpo task Admin-hpo-9559865415941081 state RUNNING progress 1/2
Hpo task Admin-hpo-9559865415941081 state RUNNING progress 1/2
Hpo task Admin-hpo-9559865415941081 completes with state FINISHED
{
    "best": {
        "appId": "Admin-9559934187167434-1923606690",
        "driverId": "driver-20191226032303-0292-75c6a513-1cc7-4984-965b-5b87eccd15c4",
        "endTime": "2019-12-26 03:24:10",
        "hyperParams": [
            {
                "dataType": "double",
                "fixedVal": "0.2",
                "name": "learning_rate",
                "type": "fix",
                "userDefined": false
          

In [115]:
getTuneStatusUrl = 'https://{}:49443/platform/rest/deeplearning/v1/hypersearch'.format(hostname)
res = req.get(getTuneStatusUrl, headers=commonHeaders, verify=False, auth=('Admin', 'Admin'))
if not res.ok:
    print('check tune job status failed: code=%s, %s'%(res.status_code, res.content))
else:
    #print(json.dumps(r.json(), sort_keys=True, indent=4))
    if len(res.json()) == 0:
        print('There is no hpo task been created')
    for item in res.json():
        #print(item['hpoName'])
        print('Hpo task: %s, State: %s'%(item['hpoName'], item['state']))
        #print('Hpo tasks detail:%s'%json.dumps(item, sort_keys=True, indent=4))

Hpo task: Admin-hpo-9551207096118021, State: FINISHED
Hpo task: Admin-hpo-9559865415941081, State: FINISHED


In [116]:
deleteHpoUrl = 'https://{}:49443/platform/rest/deeplearning/v1/hypersearch'.format(hostname)
r=req.delete(deleteHpoUrl,headers=commonHeaders, verify=False, auth=('Admin','Admin'))
if not r.ok:
    print('delete hpo task failed: code=%s, %s'%(r.status_code, r.content))
else:
    print('delete hpo task succeeds')

delete hpo task succeeds
